<a href="https://colab.research.google.com/github/tarupathak30/rag-astronomy-chatbot-/blob/main/exoplanets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q sentence-transformers faiss-cpu transformers langchain_community
!pip install -q astroquery


In [ ]:
!pip install wikipedia

In [ ]:
import wikipedia

page = wikipedia.page("Brown dwarf")


In [ ]:
print(page.content[:500])


In [ ]:
import wikipedia

keywords = [
    "Brown dwarf",
    "Gas giant",
    "Planet",
    "Star",
    "Main sequence",
    "Exoplanet",
    "Radial velocity",
    "Transit method",
    "Direct imaging",
    "Neutron star",
    "White dwarf",
    "Galaxy",
    "Dark matter",
    "Dark energy",
    "Supernova",
    "Black hole",
    "Protoplanetary disk",
    "Astrobiology"
]

concept_texts = []

for topic in keywords:
    try:
        page = wikipedia.page(topic)
        if(topic == "Brown dwarf"):
          print(page)
        concept_texts.append((topic, page.content))
    except:
        pass


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [ ]:
# ============================
# 🔭 2. IMPORTS
# ============================
import pandas as pd
import pickle
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document
from sentence_transformers import SentenceTransformer, CrossEncoder
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive

# ============================
# 🔭 3. DOWNLOAD EXOPLANET DATA
# ============================
exo = NasaExoplanetArchive.query_criteria(table="pscomppars")
exoplanets = exo.to_pandas()
exoplanets.fillna("", inplace=True)
exoplanets.to_csv("exoplanets.csv", index=False)

print("Downloaded exoplanet rows:", len(exoplanets))

# ============================
# 🔭 4. BUILD DOCUMENTS
# ============================


In [ ]:
# -------------------------------
# 3️⃣ Build system-level docs
# -------------------------------
def build_docs(exo):
    docs = []
    grouped = exo.group_by("hostname")
    for group in grouped.groups:
        star = group['hostname'][0]
        context_text = ""
        for row in group:
            context_text += (
                f"planet_name: {row['pl_name']}\n"
                f"orbital_period_days: {row['pl_orbper']}\n"
                f"mass_earth: {row['pl_bmasse']}\n"
                f"radius_earth: {row['pl_rade']}\n\n"
            )
        docs.append(Document(page_content=context_text, metadata={"host_star": star}))
    print(f"Built {len(docs)} system-level documents")
    return docs

In [ ]:
docs = build_docs(exo)

In [ ]:

# -------------------------------
# 4️⃣ Load embeddings + FAISS
# -------------------------------
def build_vector_index(docs):
    embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
    texts = [d.page_content for d in docs]
    metadatas = [d.metadata for d in docs]
    index = FAISS.from_texts(texts=texts, embedding=embedder, metadatas=metadatas)
    index.save_local("faiss_index")
    pickle.dump(docs, open("docs.pkl", "wb"))
    print("✅ FAISS index and docs saved successfully")
    return index


In [ ]:
index = build_vector_index(docs)

In [ ]:
# -------------------------------
# 6️⃣ Load LLM + tokenizer
# -------------------------------
MODEL_ID = "Qwen/Qwen2.5-1.5B-Instruct"
tok = AutoTokenizer.from_pretrained(MODEL_ID)
llm = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float32,
    device_map="auto"
)
device = next(llm.parameters()).device
print("LLM loaded on:", device)

In [ ]:
from sentence_transformers import CrossEncoder
reranker = CrossEncoder("BAAI/bge-reranker-base")

In [ ]:
def rerank_docs(query, docs):
    texts = [d.page_content for d in docs]
    pairs = [(query, t) for t in texts]
    scores = reranker.predict(pairs)
    best_idx = int(scores.argmax())
    return docs[best_idx]  # ✅ return Document object


In [ ]:
import pandas as pd

df = pd.read_csv("exoplanets.csv")


In [ ]:
cols = df.columns.tolist()
len(cols)

In [ ]:
df.loc[df['pl_name'].str.lower()=="wasp-18 b", ['pl_name','pl_orbper','pl_orbpererr1','pl_orbpererr2']]


In [ ]:
df.loc[df['pl_name'].str.lower()=="epler-1604 b", ['pl_rade', 'pl_name','pl_orbper','pl_orbpererr1','pl_orbpererr2', 'pl_bmasse']]


In [ ]:
# ================================================
# FIXED: METADATA + HELPERS + PIPELINE
# ================================================
import pandas as pd
import re
import difflib
import torch
from typing import Optional
from langchain.docstore.document import Document

# -------------------------------
# Basic helpers
# -------------------------------
def extract_number(value):
    """Extract numeric value from strings, return float or None."""
    if pd.isna(value):
        return None
    if isinstance(value, (int, float)):
        return float(value)
    s = str(value)
    # Remove commas used as thousands separators
    s = s.replace(",", "")
    match = re.findall(r"[-+]?\d*\.\d+|\d+", s)
    return float(match[0]) if match else None


def tokenize(col_name):
    """Split words for fuzzy matching."""
    return re.findall(r'\w+', str(col_name).lower())


# -------------------------------
# Robust column mapper
# -------------------------------
def map_query_to_column(query, columns):
    """
    Preferred deterministic mapping for common questions (period, mass, radius).
    Falls back to fuzzy matching only if none of the keywords are present.
    """
    q = query.lower()
    if any(k in q for k in ["orbital", "orbite", "orb", "period"]):
        if "pl_orbper" in columns:
            return "pl_orbper"
    if any(k in q for k in ["mass", "heavy", "heaviest", "most massive"]):
        if "pl_bmasse" in columns:
            return "pl_bmasse"
        if "pl_massj" in columns:
            return "pl_massj"
    if any(k in q for k in ["radius", "size", "largest radius"]):
        if "pl_rade" in columns:
            return "pl_rade"

    # fallback fuzzy matching (lower priority)
    query_toks = tokenize(q)
    best_col, best_score = None, 0
    for col in columns:
        col_toks = tokenize(col)
        for qt in query_toks:
            for ct in col_toks:
                score = difflib.SequenceMatcher(None, qt, ct).ratio()
                if score > best_score:
                    best_score = score
                    best_col = col
    return best_col


# -------------------------------
# Normalize units for orbital period
# -------------------------------
def normalize_orbital_column_to_days(series):
    """
    Attempt to normalize pl_orbper to days.
    Heuristic:
      - many values < 1 -> likely in years -> convert (years * 365.25)
      - otherwise assume already in days
    Returns a numeric Series (days) with NaNs preserved.
    """
    nums = series.map(extract_number)
    # how many numeric values < 1?
    valid = nums.dropna()
    if valid.empty:
        return nums  # nothing to do
    frac_lt1 = (valid < 1).sum() / len(valid)
    if frac_lt1 > 0.5:
        # more than half < 1 -> assume years -> convert to days
        return nums * 365.25
    return nums


# -------------------------------
# Build text context for LLM
# -------------------------------
def build_context(row):
    """Return a readable text block from a Series / dict-like row."""
    ctx = []
    if hasattr(row, "items"):
        iterator = row.items()
    else:
        iterator = row._asdict().items()
    for col, val in iterator:
        if pd.notna(val):
            ctx.append(f"{col}: {val}")
    return "\n".join(ctx)


# -------------------------------
# Get top row robustly (max/min)
# -------------------------------
def get_top_row(df, column, query=None, prefer_max=True):
    """
    Finds the top row (max or min) for column.
    - Ensures numeric conversion via extract_number
    - For pl_orbper, normalizes to days first
    - Returns full row (Series) or None
    """
    df2 = df.copy()

    # special normalization for orbital period column
    if column == "pl_orbper":
        df2["_norm_orbper_days"] = normalize_orbital_column_to_days(df2[column])
        val_col = "_norm_orbper_days"
    else:
        df2[column] = df2[column].map(extract_number)
        val_col = column

    df2 = df2.dropna(subset=[val_col])
    if df2.empty:
        return None

    # interpret query words for min/max
    if query:
        q = query.lower()
        if any(w in q for w in ["shortest", "smallest", "least"]):
            prefer_max = False

    if prefer_max:
        idx = df2[val_col].idxmax()
    else:
        idx = df2[val_col].idxmin()

    return df2.loc[idx]


# ================================================
# 2) ANSWER ENGINE (STRUCTURED + DOC + LLM) - FIXED
# ================================================
def answer_query(query, df, docs, tok, llm, device):
    """
    Fixed answer_query:
      - robust column detection
      - numeric normalization for orbital period (days)
      - only retrieve docs for the selected planet
      - safe tokenization & device movement
      - low creativity generation (temperature=0.0, no sampling)
    """
    # 1) map query -> column
    col = map_query_to_column(query, df.columns)
    if not col:
        return "No matching column found."

    # 2) get the winner row (use query to decide min/max)
    row = get_top_row(df, col, query=query, prefer_max=True)
    if row is None:
        return "No valid numeric data found for this column."

    # 3) build structured context from that single row
    structured = build_context(row)

    # 4) retrieve docs only for this planet (docs are Document objects)
    planet_name = str(row.get("pl_name", "")).lower()
    related_docs = []
    for d in docs:
        # d may be a Document or a plain string — handle both
        text = d.page_content if isinstance(d, Document) else str(d)
        if planet_name and planet_name in text.lower():
            related_docs.append(text)
    top_doc = related_docs[0] if related_docs else "No retrieved documents available."

    # 5) build the prompt (short & strict to avoid hallucination)
    prompt = (
        "You are an astronomy assistant. Use ONLY the data below. Do NOT invent or combine planets.\n\n"
        "Structured data (single selected row):\n"
        f"{structured}\n\n"
        "System-level data (about the same planet):\n"
        f"{top_doc}\n\n"
        f"Question: {query}\n\n"
        "Final Answer:"
    )

    # 6) tokenize safely (tokenizer returns a dict of tensors)
    inputs = tok(prompt, return_tensors="pt")
    # move tensors to device safely
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # 7) generate (low temperature, deterministic)
    with torch.no_grad():
        # prefer passing explicit input_ids and attention_mask if present
        gen_kwargs = {
            "input_ids": inputs.get("input_ids"),
            "max_new_tokens": 150,
            "temperature": 0.0,
            "do_sample": False
        }
        if "attention_mask" in inputs:
            gen_kwargs["attention_mask"] = inputs["attention_mask"]

        outputs = llm.generate(**gen_kwargs)

    # 8) decode and extract final answer
    text = tok.decode(outputs[0], skip_special_tokens=True)
    if "Final Answer:" in text:
        text = text.split("Final Answer:")[-1].strip()
    return text


In [ ]:
result = answer_query(
    "Which exoplanet has the longest orbital period?",
    df,
    docs,
    tok,
    llm,
    device
)

print(result)
